In [1]:
import re
import gc
import pickle
import argparse
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.layers import *
from keras import optimizers
from keras import backend as K
from collections import Counter
from keras.utils import np_utils
from gensim.models import word2vec
from sklearn.metrics import f1_score
from keras.regularizers import l1, l2
from keras.engine import InputSpec, Layer
from keras.models import Model, load_model
from keras.optimizers import RMSprop, Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import text, sequence
from keras.layers import Input, Embedding, Dense
from sklearn.preprocessing import StandardScaler
from keras import optimizers, losses, activations
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tqdm import tqdm
from keras.layers.noise import GaussianNoise
from models_def import Attention
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from keras.callbacks import LearningRateScheduler
config = argparse.Namespace()

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
der=True
if der:
    train_dir = '../input/new_data/train_set.csv'
    test_dir = '../input/new_data/test_set.csv'
else:
#     train_dir = '../input/new_data/train_remove60.csv'
#     test_dir = '../input/new_data/test_remove60.csv'
    train_dir = '../input/new_data/train_pre_enhance.csv'
    test_dir = '../input/new_data/test_set.csv'
train = pd.read_csv(train_dir)
test = pd.read_csv(test_dir)
original_train = pd.read_csv('../input/new_data/train_set.csv')
shuffle_data = train[original_train.shape[0]:]

In [3]:
all_data = pd.concat([train, test])
all_data = all_data.reset_index(drop=True)
train_offset = train.shape[0]

In [4]:
df_y_train = (train["class"]-1).astype(int)
# column = "word_seg"
test_id = test[["id"]].copy()

/usr/local/lib/python3.5/dist-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [5]:
column = "word_seg"
config.len_desc = 800000
tknzr_word = Tokenizer(num_words=config.len_desc)
tknzr_word.fit_on_texts(all_data[column].values)

In [6]:
count_thres = 5
low_count_words = [w for w, c in tknzr_word.word_counts.items() if c < count_thres]
for w in tqdm(low_count_words):
    del tknzr_word.word_index[w]
    del tknzr_word.word_docs[w]
    del tknzr_word.word_counts[w]

100%|██████████| 912182/912182 [00:01<00:00, 749452.81it/s]


In [7]:
tr_word_seq = tknzr_word.texts_to_sequences(train[column].values)
te_word_seq = tknzr_word.texts_to_sequences(test[column].values)
config.maxlen = 300
tr_word_pad = pad_sequences(tr_word_seq, maxlen=config.maxlen)
te_word_pad = pad_sequences(te_word_seq, maxlen=config.maxlen)

In [8]:
tr_word_post = pad_sequences(tr_word_seq, maxlen=config.maxlen, padding='post', truncating='post')
print('Shape of data tensor:', tr_word_post.shape)
# print('Shape of label tensor:', y.shape)

te_word_post = pad_sequences(te_word_pad, maxlen=config.maxlen, padding='post', truncating='post')
# print('Shape of test_data tensor:', test_data_post.shape)

Shape of data tensor: (102277, 300)


In [9]:
from gensim.models import word2vec
max_features = 800000
vec_len = 600
EMBEDDING = '../feature/word2vec_file/avito600d.w2v'
model = word2vec.Word2Vec.load(EMBEDDING)
word_index = tknzr_word.word_index
nb_words_desc = min(max_features, len(word_index))
embedding_matrix_desc = np.zeros((nb_words_desc+1, vec_len))
for word, i in word_index.items():
    if i >= max_features: continue
    try:
        embedding_vector = model[word]
    except KeyError:
        embedding_vector = None
    if embedding_vector is not None: embedding_matrix_desc[i] = embedding_vector

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


In [10]:
# column = "article"
# config.len_title = 100000
# tknzr_article = Tokenizer(num_words=config.len_title)
# tknzr_article.fit_on_texts(all_data[column].values)

In [11]:
# count_thres = 4
# low_count_words = [w for w, c in tknzr_article.word_counts.items() if c < count_thres]
# # print(len(tknzr_word.texts_to_sequences(all_data[column].values)))
# for w in tqdm(low_count_words):
#     del tknzr_article.word_index[w]
#     del tknzr_article.word_docs[w]
#     del tknzr_article.word_counts[w]

In [12]:
# tr_article_seq = tknzr_article.texts_to_sequences(train[column].values)
# te_article_seq = tknzr_article.texts_to_sequences(test[column].values)
# config.titlemaxlen= 300
# tr_article_pad = pad_sequences(tr_article_seq, maxlen=config.titlemaxlen)
# te_article_pad = pad_sequences(te_article_seq, maxlen=config.titlemaxlen)
# del all_data
# gc.collect()
# article_index = tknzr_article.word_index
# nb_article = min(max_features, len(article_index))

In [13]:
# max_features = 500000
# vec_len = 600
# EMBEDDING = "../feature/word2vec_file/avito600d.w2v"
# model = word2vec.Word2Vec.load(EMBEDDING)
# char_embedding_matrix = np.zeros((nb_article+1, vec_len))
# for word, i in article_index.items():
#     if i >= max_features: continue
#     try:
#         embedding_vector = model[word]
#     except KeyError:
#         embedding_vector = None
#     if embedding_vector is not None: char_embedding_matrix[i] = embedding_vector

In [14]:
Y = df_y_train 

In [15]:
def squash(x, axis=-1):
    # s_squared_norm is really small
    # s_squared_norm = K.sum(K.square(x), axis, keepdims=True) + K.epsilon()
    # scale = K.sqrt(s_squared_norm)/ (0.5 + s_squared_norm)
    # return scale * x
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True)
    scale = K.sqrt(s_squared_norm + K.epsilon())
    return x / scale
from keras.engine import Layer, InputSpec
class Capsule(Layer):
    def __init__(self, num_capsule, dim_capsule, routings=3, kernel_size=(9, 1), share_weights=True,
                 activation='default', **kwargs):
        super(Capsule, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.kernel_size = kernel_size
        self.share_weights = share_weights
        if activation == 'default':
            self.activation = squash
        else:
            self.activation = Activation(activation)

    def build(self, input_shape):
        super(Capsule, self).build(input_shape)
        input_dim_capsule = input_shape[-1]
        if self.share_weights:
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(1, input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     # shape=self.kernel_size,
                                     initializer='glorot_uniform',
                                     trainable=True)
        else:
            input_num_capsule = input_shape[-2]
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(input_num_capsule,
                                            input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     initializer='glorot_uniform',
                                     trainable=True)

    def call(self, u_vecs):
        if self.share_weights:
            u_hat_vecs = K.conv1d(u_vecs, self.W)
        else:
            u_hat_vecs = K.local_conv1d(u_vecs, self.W, [1], [1])

        batch_size = K.shape(u_vecs)[0]
        input_num_capsule = K.shape(u_vecs)[1]
        u_hat_vecs = K.reshape(u_hat_vecs, (batch_size, input_num_capsule,
                                            self.num_capsule, self.dim_capsule))
        u_hat_vecs = K.permute_dimensions(u_hat_vecs, (0, 2, 1, 3))
        # final u_hat_vecs.shape = [None, num_capsule, input_num_capsule, dim_capsule]

        b = K.zeros_like(u_hat_vecs[:, :, :, 0])  # shape = [None, num_capsule, input_num_capsule]
        for i in range(self.routings):
            b = K.permute_dimensions(b, (0, 2, 1))  # shape = [None, input_num_capsule, num_capsule]
            c = K.softmax(b)
            c = K.permute_dimensions(c, (0, 2, 1))
            b = K.permute_dimensions(b, (0, 2, 1))
            outputs = self.activation(K.batch_dot(c, u_hat_vecs, [2, 2]))
            if i < self.routings - 1:
                b = K.batch_dot(outputs, u_hat_vecs, [2, 3])

        return outputs

    def compute_output_shape(self, input_shape):
        return (None, self.num_capsule, self.dim_capsule)


In [16]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [17]:
def get_model():
    filter_nr = 128
    filter_size = 3
    max_pool_size = 3
    max_pool_strides = 2
    dense_nr = 256
    spatial_dropout = 0.4
    dense_dropout = 0.4
    
    inpword = Input(shape=(1000, ))
    inparticle = Input(shape=(1000, ))
    

    emb_comment1 = Embedding(embedding_matrix_desc.shape[0], 600,  weights = [embedding_matrix_desc],trainable=False)(inpword)
#     emb_comment1 = Embedding(nb_words, EMBEDDING_DIM, weights=[embedding_matrix],trainable=False)(inpword)
    emb_comment1 = SpatialDropout1D(spatial_dropout)(emb_comment1)

    block11 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(emb_comment1)
    block11 = BatchNormalization()(block11)
    block11 = PReLU()(block11)
    block11 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(block11)
    block11 = BatchNormalization()(block11)
    block11 = PReLU()(block11)

    resize_emb1 = Conv1D(filter_nr, kernel_size=1, padding='same', activation='linear')(emb_comment1)
    resize_emb1 = PReLU()(resize_emb1)

    block1_output1 = add([block11, resize_emb1])
    block1_output1 = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(block1_output1)

    block21 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(block1_output1)
    block21 = BatchNormalization()(block21)
    block21 = PReLU()(block21)
    block21 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(block21)
    block21 = BatchNormalization()(block21)
    block21 = PReLU()(block21)

    block2_output1 = add([block21, block1_output1])
    block2_output1 = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(block2_output1)

    block31 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(block2_output1)
    block31 = BatchNormalization()(block31)
    block31 = PReLU()(block31)
    block31 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(block31)
    block31 = BatchNormalization()(block31)
    block31 = PReLU()(block31)

    block3_output1 = add([block31, block2_output1])
    block3_output1 = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(block3_output1)

    block41 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(block3_output1)
    block41 = BatchNormalization()(block41)
    block41 = PReLU()(block41)
    block41 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(block41)
    block41 = BatchNormalization()(block41)
    block41 = PReLU()(block41)

    output1 = add([block41, block3_output1])
    output1 =GlobalMaxPooling1D()(output1)


    emb_comment2 =  Embedding(embedding_matrix_desc.shape[0], 600,  weights = [embedding_matrix_desc],trainable=False)(inparticle)
    emb_comment2 = SpatialDropout1D(spatial_dropout)(emb_comment2)

    block12 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(emb_comment2)
    block12 = BatchNormalization()(block12)
    block12 = PReLU()(block12)
    block12 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(block12)
    block12 = BatchNormalization()(block12)
    block12 = PReLU()(block12)

    resize_emb2 = Conv1D(filter_nr, kernel_size=1, padding='same', activation='linear')(emb_comment2)
    resize_emb2 = PReLU()(resize_emb2)

    block1_output2 = add([block12, resize_emb2])
    block1_output2 = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(block1_output2)

    block22 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(block1_output2)
    block22 = BatchNormalization()(block22)
    block22 = PReLU()(block22)
    block22 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(block22)
    block22 = BatchNormalization()(block22)
    block22 = PReLU()(block22)

    block2_output2 = add([block22, block1_output2])
    block2_output2 = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(block2_output2)

    block32 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(block2_output2)
    block32 = BatchNormalization()(block32)
    block32 = PReLU()(block32)
    block32 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(block32)
    block32 = BatchNormalization()(block32)
    block32 = PReLU()(block32)

    block3_output2 = add([block32, block2_output2])
    block3_output2 = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(block3_output2)

    block42 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(block3_output2)
    block42 = BatchNormalization()(block42)
    block42 = PReLU()(block42)
    block42 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(block42)
    block42 = BatchNormalization()(block42)
    block42 = PReLU()(block42)

    output2 = add([block42, block3_output2])
    output2 = GlobalMaxPooling1D()(output2)

            
    output = concatenate([output1, output2])
    output = Dense(dense_nr, activation='linear')(output)
    output = BatchNormalization()(output)
    output = PReLU()(output)
    output = Dropout(dense_dropout)(output)
    x = Dense(19, activation="softmax")(output)
    model = Model(inputs=[inpword, inparticle], outputs=x)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                    metrics=[f1])
    
    return model

def get_Capsule_model():
#     features_input = Input(shape=(features.shape[1],))

    inpword = Input(shape=(config.maxlen, ))
    emb_word = Embedding(embedding_matrix_desc.shape[0], 600,  weights = [embedding_matrix_desc],trainable=False)(inpword)
    embed_layer = SpatialDropout1D(0.4)(emb_word)

    x = Bidirectional(CuDNNGRU(128, return_sequences=True))(embed_layer)
    x = Bidirectional(CuDNNGRU(128, return_sequences=True))(x)
    capsule = Capsule(num_capsule=10, dim_capsule=32, routings=5,share_weights=True)(x)
    capsule = Flatten()(capsule)
    x = Dropout(0.2)(capsule)
    x = PReLU()(Dense(128)(x))
    
    
    x = Dense(19, activation="softmax")(x)
    model = Model(inputs=[inpword], outputs=x)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                    metrics=[f1])
    
    return model

def deepmoji():
    maxlen = 300
    filter_sizes = [1,2,3,5]
    num_filters = 16
    inpword = Input(shape=(300, ))
    inparticle = Input(shape=(300, ))
    embed_l2 = 0
    embed_reg = L1L2(l2=embed_l2) if embed_l2 != 0 else None
    x1 = Embedding(embedding_matrix_desc.shape[0], 600,  weights = [embedding_matrix_desc],trainable=False, embeddings_regularizer=embed_reg)(inpword)
    
    x1 = SpatialDropout1D(0.4)(x1)
    x1 = Reshape((config.maxlen, 600,  1))(x1)
    
    pooled_pre = []
    for i in filter_sizes:
    
        conv_pre = Conv2D(num_filters, kernel_size=(i, 300), kernel_initializer='normal',
                    activation='elu')(x1)
    
        maxpool_pre = MaxPool2D(pool_size=(maxlen - i + 1, 1))(conv_pre)
        avepool_pre = AveragePooling2D(pool_size=(maxlen - i + 1, 1))(conv_pre)
        globalmax_pre = GlobalMaxPooling2D()(conv_pre)
        pooled_pre.append(globalmax_pre)
        
    z1 = Concatenate(axis=1)(pooled_pre)   
    z1 = Dropout(0.2)(z1)
    
#     x1_post = Embedding(embedding_matrix_desc.shape[0], 600,  weights = [embedding_matrix_desc],trainable=False, embeddings_regularizer=embed_reg)(inparticle)
#     x1_post = SpatialDropout1D(0.4)(x1_post)
#     x1_post = Reshape((config.maxlen, 600,  1))(x1_post)
    
#     pooled_post = []
#     for i in filter_sizes:

#         conv_post = Conv2D(num_filters, kernel_size=(i, 300), kernel_initializer='normal',
#                     activation='elu')(x1_post)
    
#         maxpool_post = MaxPool2D(pool_size=(maxlen - i + 1, 1))(conv_post)
#         avepool_post = AveragePooling2D(pool_size=(maxlen - i + 1, 1))(conv_post)
#         globalmax_post = GlobalMaxPooling2D()(conv_post)
#         pooled_post.append(globalmax_post)
        
#     z1_post = Concatenate(axis=1)(pooled_post)   
#     z1_post = Dropout(0.2)(z1_post)
#     z = concatenate([z1,z1_post])
    x = Dense(19, activation="softmax")(z1)
    model = Model(inputs=[inpword, inparticle], outputs=x)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                    metrics=[f1])
    
    return model

In [18]:
one_y_train=np_utils.to_categorical(Y,num_classes=19)
one_y_val=np_utils.to_categorical(Y,num_classes=19)

In [19]:
decay_dic = {0:0.001, 1:0.001, 2:0.0009, 3:0.0008, 4:0.0007, 5:0.0006, 6:0.0005, 7:0.0004, 8:0.0003, 9:0.0002, 10:0.0001,
            11:0.00009, 12:0.00008, 13:0.00007, 14:0.00006, 15:0.00005}
def lr_decay(epoch):
    return decay_dic[epoch]
from sklearn.model_selection import KFold
def kf_train(fold_cnt=3, rnd=1):
    now_nfold = 0
    kf = KFold(n_splits=fold_cnt, shuffle=False, random_state=233 * rnd)
    train_pred, test_pred = np.zeros((102277, 19)), np.zeros((102277, 19))
    LRDecay = LearningRateScheduler(lr_decay)
    for train_index, test_index in kf.split(train):
        # x,y
        now_nfold += 1
        print("now is {} fold".format(now_nfold))

        curr_x1, curr_x2 = tr_word_pad[train_index], tr_word_post[train_index]
        hold_out_x1, hold_out_x2 = tr_word_pad[test_index], tr_word_post[test_index]
#         curr_x1 = tr_word_pad[train_index]
#         hold_out_x1 = tr_word_pad[test_index]
        curr_y, hold_out_y = one_y_train[train_index], one_y_train[test_index]

#         kfold_X_features = features[train_index]
#         kfold_X_valid_features = features[test_index]

        config.batch_size=32
        epochs = 15

        model = deepmoji()

        file_path = "weights_base.best.h5"
        checkpoint = ModelCheckpoint(file_path, save_best_only=True, verbose=1, monitor='val_f1', mode='max')
        early = EarlyStopping(monitor='val_f1', mode='max', patience=2, )
        callbacks_list = [checkpoint, early, LRDecay]

        model.fit([curr_x1, curr_x2], curr_y,
                  batch_size=config.batch_size, epochs=epochs,
                  validation_data=([hold_out_x1,hold_out_x2], hold_out_y),
                  callbacks=callbacks_list)


        model.load_weights(file_path)

        y_test = model.predict([te_word_pad, te_word_post])
        test_pred += y_test
        hold_out_pred = model.predict([hold_out_x1, hold_out_x2])
        train_pred[test_index] = hold_out_pred

        del model
        gc.collect()
        K.clear_session()
    test_pred = test_pred / fold_cnt
    print('-------------------------------')
    try:
        print('all eval', sqrt(mean_squared_error(Y, train_pred)))
    finally:
        return train_pred, test_pred

In [20]:
import time
import pickle
model_time = time.time()
print("start")
train_pred,test_pred = kf_train(fold_cnt=10,rnd=4)
print (train_pred.shape,test_pred.shape)
print ("[{}] finished nn model".format((time.time()-model_time)/3600))

start
now is 1 fold
Train on 92049 samples, validate on 10228 samples
Epoch 1/15
 5152/92049 [>.............................] - ETA: 55:28 - loss: 10.5581 - f1: 0.0540

KeyboardInterrupt: 

In [ ]:
with open('../feature/dpcnn1.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

In [ ]:
test_pred[0]